## Declare constants

In [1]:
capture_name = 'obama_interview'
LAPLACIAN_THRESHOLD = 20
target_frames_per_video = 30
flip = False

In [2]:
import os
root_dir = os.path.join('./test_reenact',capture_name)
video_path = os.path.join(root_dir, 'video.mp4')
rgb_dir = os.path.join(root_dir, 'rgb')

## Flatten into images and remove blurry images

In [3]:
import cv2
import os
def flatten_video(video_path, rgb_dir, target_frames_per_video):
    # Create directories
    if not os.path.exists(rgb_dir):
        os.makedirs(rgb_dir)
    print('Flattening {} to {}'.format(video_path, rgb_dir))
    cap = cv2.VideoCapture(str(video_path))
    input_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if frame_count < target_frames_per_video:
        raise RuntimeError('The video is too short and has fewer frames than the target.')
    # hyper-parameters
    success,image = cap.read()
    count = 0
    skip = round(frame_count / target_frames_per_video)
    skip_count = 0
    print('Input FPS: {}'.format(input_fps))
    print('Frame count: {}'.format(frame_count))
    print('Skip: {}'.format(skip))
    blurry_count = 0
    while success:
        # get frame from video
        success,image = cap.read()
        if not success:
            break
        if skip_count < skip:
            skip_count += 1
            continue
        skip_count = 0
        if cv2.Laplacian(image, cv2.CV_64F).var() < LAPLACIAN_THRESHOLD: # check if image is blurry, if so, skip this frame
            blurry_count += 1
            continue
        if flip:
            image = cv2.flip(image, -1)
        cv2.imwrite(os.path.join(rgb_dir,f"{count:06d}.png"),image)
        count += 1
    print('Failed to read a frame, current count =', count)
    print('Blurry count: {}'.format(blurry_count))


In [4]:
# process three videos
flatten_video(video_path, rgb_dir, target_frames_per_video)

Flattening ./test_reenact/obama_interview/video.mp4 to ./test_reenact/obama_interview/rgb
Input FPS: 29.97002997002997
Frame count: 460
Skip: 15
Failed to read a frame, current count = 28
Blurry count: 0
